In [138]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import yfinance as yf
import numpy as np
from io import StringIO
import sklearn as skn
from scipy import stats
import pandas_datareader.data as pdr
import riskfolio as rp
from scipy.stats import multivariate_normal


In [2]:
def loadStooqData(ticker: str,start, frequency='d'):
    url = f'https://stooq.pl/q/d/l/?s={ticker}&i=d'
    response = requests.get(url)
    data = pd.read_csv(StringIO(response.text))
    data.set_index('Data', inplace=True)
    data = data[start:]
    return data['Zamkniecie']

def loadYahooData(ticker, start, frequency = '1d'):
    ticker  = yf.Ticker(ticker)
    hist = ticker.history(start=start,interval=frequency)
    price = hist['Close']
    return price

def sharp(returns):
    return returns.mean()/returns.std()

def assets_performance(returns: pd.DataFrame):
    return returns.agg(['mean', 'std', 'median', 'skew', 'kurtosis', sharp])

def beta(X, Y):

    X_cov = X.cov()

    XY = np.append(X,np.expand_dims(Y,axis=1),axis=0)
    XY_cov = np.cov(XY)

    #XY_sub = XY_cov.iloc[-1,0:2]

    #inv = np.linalg.inv(X_cov)
    #B = inv @ XY_sub
    return  XY_cov

In [3]:
yahoo_tickers = ['ISAC.L', 'CORP.L']

stooq_tickers = ['XAUPLN', '^TBSP', 'MWIG40TR', 'USDPLN', 'EURPLN', 'PLOPLN3M']

#na przyszłość EAFA,EEM,ACWI



In [139]:
base_prices = pd.DataFrame({})
start = '2012-06-01'

for ticker in stooq_tickers:
    base_prices[ticker] = loadStooqData(ticker,start)

for ticker in yahoo_tickers:
    base_prices[ticker] = loadYahooData(ticker,start)

base_prices.index = pd.DatetimeIndex(base_prices.index)


In [140]:
#get monthly data
monthly_base_prices = base_prices.resample('m').last()

In [141]:
monthly_base_returns = np.log(monthly_base_prices/monthly_base_prices.shift(1))
assets_performance(monthly_base_returns)

,XAUPLN,^TBSP,MWIG40TR,USDPLN,EURPLN,PLOPLN3M,ISAC.L,CORP.L
mean,0.002930,0.002461,0.008445,0.001493,0.000409,0.002037,0.007377,-0.001206
std,0.042878,0.013722,0.050936,0.029207,0.015186,0.174314,0.038651,0.019728
median,0.000431,0.002713,0.011573,0.002273,-0.000887,0.000000,0.012231,0.001964
skew,-0.020600,0.325552,-0.676854,-0.083141,0.455681,2.554163,-0.609487,-0.696820
kurtosis,0.558847,5.222528,3.301922,-0.608105,0.112551,28.371728,1.011532,2.579226
sharp,0.068332,0.179352,0.165805,0.051116,0.026956,0.011686,0.190863,-0.061112


In [142]:
monthly_base_returns.corr()

,XAUPLN,^TBSP,MWIG40TR,USDPLN,EURPLN,PLOPLN3M,ISAC.L,CORP.L
XAUPLN,1.000000,0.070347,-0.169383,0.364011,0.368558,0.026604,-0.243667,0.032810
^TBSP,0.070347,1.000000,0.092790,-0.237859,-0.147964,-0.410850,0.207925,0.546014
MWIG40TR,-0.169383,0.092790,1.000000,-0.462117,-0.501271,-0.054300,0.731645,0.553636
USDPLN,0.364011,-0.237859,-0.462117,1.000000,0.719053,0.079380,-0.549574,-0.594150
EURPLN,0.368558,-0.147964,-0.501271,0.719053,1.000000,0.023200,-0.504838,-0.420330
PLOPLN3M,0.026604,-0.410850,-0.054300,0.079380,0.023200,1.000000,-0.051869,-0.118054
ISAC.L,-0.243667,0.207925,0.731645,-0.549574,-0.504838,-0.051869,1.000000,0.645018
CORP.L,0.032810,0.546014,0.553636,-0.594150,-0.420330,-0.118054,0.645018,1.000000


In [143]:
#calculate pln returns
pln_prices = monthly_base_prices
pln_prices['ISAC.L'] = monthly_base_prices['ISAC.L'] * monthly_base_prices['USDPLN']
pln_prices['CORP.L'] = monthly_base_prices['CORP.L'] * monthly_base_prices['USDPLN']
pln_returns = np.log(pln_prices/pln_prices.shift(1))
pln_returns['PLOPLN3M'] = pln_prices['PLOPLN3M'] / 100 / 12
assets_performance(pln_returns)

,XAUPLN,^TBSP,MWIG40TR,USDPLN,EURPLN,PLOPLN3M,ISAC.L,CORP.L
mean,0.002930,0.002461,0.008445,0.001493,0.000409,0.002135,0.008870,0.000651
std,0.042878,0.013722,0.050936,0.029207,0.015186,0.001640,0.033259,0.023705
median,0.000431,0.002713,0.011573,0.002273,-0.000887,0.001442,0.007464,0.001258
skew,-0.020600,0.325552,-0.676854,-0.083141,0.455681,1.256210,-0.039220,0.239377
kurtosis,0.558847,5.222528,3.301922,-0.608105,0.112551,0.649265,0.525188,-0.363548
sharp,0.068332,0.179352,0.165805,0.051116,0.026956,1.301651,0.266696,0.027446


In [144]:
pln_returns.corr()

,XAUPLN,^TBSP,MWIG40TR,USDPLN,EURPLN,PLOPLN3M,ISAC.L,CORP.L
XAUPLN,1.000000,0.070347,-0.169383,0.364011,0.368558,-0.061628,0.036495,0.487753
^TBSP,0.070347,1.000000,0.092790,-0.237859,-0.147964,0.099145,0.032752,0.162548
MWIG40TR,-0.169383,0.092790,1.000000,-0.462117,-0.501271,-0.027093,0.444445,-0.106193
USDPLN,0.364011,-0.237859,-0.462117,1.000000,0.719053,-0.050862,0.239504,0.742882
EURPLN,0.368558,-0.147964,-0.501271,0.719053,1.000000,-0.092522,0.044771,0.553267
PLOPLN3M,-0.061628,0.099145,-0.027093,-0.050862,-0.092522,1.000000,-0.143846,-0.150598
ISAC.L,0.036495,0.032752,0.444445,0.239504,0.044771,-0.143846,1.000000,0.482816
CORP.L,0.487753,0.162548,-0.106193,0.742882,0.553267,-0.150598,0.482816,1.000000


In [145]:
pln_returns.mean() * 12

XAUPLN      0.035159
^TBSP       0.029532
MWIG40TR    0.101345
USDPLN      0.017915
EURPLN      0.004912
PLOPLN3M    0.025621
ISAC.L      0.106440
CORP.L      0.007807
dtype: float64

In [146]:
expected_returns = pd.Series({
'ISAC.L': 0.085/12,
'^TBSP': 0.035/12,
'XAUPLN': 0.054/12,
'MWIG40TR': 0.080/12,
'CORP.L': 0.037/12,
'PLOPLN3M': 0.03/12
})





In [147]:
assets = ['ISAC.L', '^TBSP', 'XAUPLN', 'MWIG40TR', 'CORP.L', 'PLOPLN3M']
#
cov_matrix = pln_returns[assets].cov()

* load data
* describe data in local currency
* transform data do pln
* describe data in pln

In [148]:
portfolio = rp.Portfolio(pd.DataFrame(pln_returns[assets]))
#pln_returns[assets]

portfolio.mu = expected_returns
portfolio.cov = cov_matrix

model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
rm = 'MV' # Risk measure used, this time will be variance
obj = 'MinRisk' # Objective function, could be MinRisk, MaxRet, Utility or Sharpe
hist = False # Use historical scenarios for risk measures that depend on scenarios
rf = 0 # Risk free rate
l = 0 # Risk aversion factor, only useful when obj is 'Utility'

w = portfolio.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)

display(w.T)

,ISAC.L,^TBSP,XAUPLN,MWIG40TR,CORP.L,PLOPLN3M
weights,0.004539,2.738961e-08,0.000758,0.001174,0.011144,0.982385


In [149]:
w = portfolio.optimization(model=model, rm=rm, obj='MaxRet', rf=rf, l=l, hist=hist)

display(np.round(w.T,2))

,ISAC.L,^TBSP,XAUPLN,MWIG40TR,CORP.L,PLOPLN3M
weights,1.0,0.0,0.0,0.0,0.0,0.0


In [150]:
def frontier_performance(frontier,mu,cov):
    result = frontier.copy()
    result['Returns'] = np.round(frontier.to_numpy() @ mu *12,4)
    result['Std. deviations'] = frontier.apply(lambda x: np.round(np.sqrt(x.to_numpy().T @ cov @ x.to_numpy())* np.sqrt(12),4),axis=1)
    result['SR'] = np.round(result['Returns'] / result['Std. deviations'],2)
    return result


In [151]:
def shrinkedCovariance(returns: pd.DataFrame, w: int):
    std = np.diag(returns.std())
    corr = returns.corr('pearson')
    shrinked_corr = (1-w)*corr + np.ones_like(corr)*w
    result = std @ shrinked_corr @ std
    return result

shrin_param = 0.75

In [152]:
points = 10
frontier = portfolio.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)

frontier_10 = np.round(frontier.T,2)

In [153]:
frontier_performance(frontier_10, expected_returns,cov_matrix)

,ISAC.L,^TBSP,XAUPLN,MWIG40TR,CORP.L,PLOPLN3M,Returns,Std. deviations,SR
0,0.00,0.00,0.00,0.00,0.01,0.98,0.0298,0.0055,5.42
1,0.13,0.04,0.04,0.02,0.00,0.77,0.0393,0.0180,2.18
2,0.21,0.07,0.06,0.04,0.00,0.61,0.0450,0.0295,1.53
3,0.30,0.11,0.09,0.05,0.00,0.45,0.0517,0.0419,1.23
4,0.38,0.14,0.11,0.07,0.00,0.29,0.0574,0.0536,1.07
5,0.47,0.17,0.14,0.09,0.00,0.14,0.0649,0.0669,0.97
6,0.55,0.18,0.17,0.10,0.00,0.00,0.0702,0.0780,0.90
7,0.65,0.04,0.19,0.12,0.00,0.00,0.0765,0.0908,0.84
8,0.80,0.00,0.09,0.11,0.00,0.00,0.0817,0.1032,0.79
9,1.00,0.00,0.00,0.00,0.00,0.00,0.0850,0.1152,0.74


In [154]:
mu = expected_returns
shrinked_cov = shrinkedCovariance(pln_returns[assets],shrin_param)

portfolio.mu = mu
portfolio.cov = shrinked_cov

frontier = portfolio.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)

frontier_10_sh = np.round(frontier.T,2)

In [155]:
frontier_performance(frontier_10_sh,mu,shrinked_cov)


,ISAC.L,^TBSP,XAUPLN,MWIG40TR,CORP.L,PLOPLN3M,Returns,Std. deviations,SR
0,0.00,0.0,0.0,0.0,0.0,1.00,0.0300,0.0057,5.26
1,0.12,0.0,0.0,0.0,0.0,0.88,0.0366,0.0177,2.07
2,0.23,0.0,0.0,0.0,0.0,0.77,0.0426,0.0298,1.43
3,0.34,0.0,0.0,0.0,0.0,0.66,0.0487,0.0419,1.16
4,0.45,0.0,0.0,0.0,0.0,0.55,0.0548,0.0541,1.01
5,0.56,0.0,0.0,0.0,0.0,0.44,0.0608,0.0663,0.92
6,0.67,0.0,0.0,0.0,0.0,0.33,0.0669,0.0785,0.85
7,0.78,0.0,0.0,0.0,0.0,0.22,0.0729,0.0908,0.80
8,0.89,0.0,0.0,0.0,0.0,0.11,0.0790,0.1030,0.77
9,1.00,0.0,0.0,0.0,0.0,0.00,0.0850,0.1152,0.74


In [156]:
mu = expected_returns
cov = pln_returns[0:60][assets].cov()

portfolio.mu = mu
portfolio.cov = cov

frontier = portfolio.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)

frontier_1_5 = np.round(frontier.T,2)

In [157]:
frontier_performance(frontier_1_5,mu,cov)

,ISAC.L,^TBSP,XAUPLN,MWIG40TR,CORP.L,PLOPLN3M,Returns,Std. deviations,SR
0,0.00,0.00,0.00,0.00,0.0,1.00,0.0300,0.0029,10.34
1,0.10,0.08,0.01,0.05,0.0,0.76,0.0386,0.0141,2.74
2,0.18,0.15,0.02,0.09,0.0,0.56,0.0456,0.0250,1.82
3,0.26,0.23,0.03,0.13,0.0,0.35,0.0527,0.0361,1.46
4,0.34,0.30,0.04,0.17,0.0,0.14,0.0594,0.0471,1.26
5,0.42,0.31,0.05,0.21,0.0,0.00,0.0661,0.0577,1.15
6,0.51,0.17,0.06,0.26,0.0,0.00,0.0733,0.0693,1.06
7,0.60,0.03,0.07,0.30,0.0,0.00,0.0798,0.0807,0.99
8,0.82,0.00,0.00,0.18,0.0,0.00,0.0841,0.0919,0.92
9,1.00,0.00,0.00,0.00,0.0,0.00,0.0850,0.1028,0.83


In [158]:
mu = expected_returns
shrinked_cov = shrinkedCovariance(pln_returns[0:60][assets],shrin_param)

portfolio.mu = mu
portfolio.cov = shrinked_cov

frontier = portfolio.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)

frontier_1_5_sh = np.round(frontier.T,2)

In [159]:
frontier_performance(frontier_1_5_sh,mu,shrinked_cov)

,ISAC.L,^TBSP,XAUPLN,MWIG40TR,CORP.L,PLOPLN3M,Returns,Std. deviations,SR
0,0.00,0.0,0.0,0.0,0.0,1.00,0.0300,0.0029,10.34
1,0.12,0.0,0.0,0.0,0.0,0.88,0.0366,0.0144,2.54
2,0.23,0.0,0.0,0.0,0.0,0.77,0.0426,0.0254,1.68
3,0.34,0.0,0.0,0.0,0.0,0.66,0.0487,0.0365,1.33
4,0.45,0.0,0.0,0.0,0.0,0.55,0.0548,0.0475,1.15
5,0.56,0.0,0.0,0.0,0.0,0.44,0.0608,0.0586,1.04
6,0.67,0.0,0.0,0.0,0.0,0.33,0.0669,0.0696,0.96
7,0.78,0.0,0.0,0.0,0.0,0.22,0.0729,0.0807,0.90
8,0.89,0.0,0.0,0.0,0.0,0.11,0.0790,0.0918,0.86
9,1.00,0.0,0.0,0.0,0.0,0.00,0.0850,0.1028,0.83


In [160]:
mu = expected_returns
cov = pln_returns[60:120][assets].cov()

portfolio.mu = mu
portfolio.cov = cov

frontier = portfolio.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)

frontier_5_10 = np.round(frontier.T,2)

In [161]:
frontier_performance(frontier_5_10,mu,cov)

,ISAC.L,^TBSP,XAUPLN,MWIG40TR,CORP.L,PLOPLN3M,Returns,Std. deviations,SR
0,0.01,0.05,0.00,0.00,0.0,0.94,0.0308,0.0030,10.27
1,0.12,0.18,0.05,0.01,0.0,0.64,0.0392,0.0165,2.38
2,0.21,0.30,0.09,0.02,0.0,0.38,0.0462,0.0302,1.53
3,0.30,0.42,0.13,0.02,0.0,0.13,0.0527,0.0429,1.23
4,0.41,0.40,0.17,0.03,0.0,0.00,0.0604,0.0577,1.05
5,0.51,0.25,0.20,0.03,0.0,0.00,0.0653,0.0701,0.93
6,0.61,0.12,0.24,0.03,0.0,0.00,0.0714,0.0837,0.85
7,0.72,0.00,0.25,0.03,0.0,0.00,0.0771,0.0977,0.79
8,0.89,0.00,0.11,0.00,0.0,0.00,0.0816,0.1116,0.73
9,1.00,0.00,0.00,0.00,0.0,0.00,0.0850,0.1247,0.68


In [162]:
shrinked_cov = shrinkedCovariance(pln_returns[60:120][assets],shrin_param)

portfolio.mu = mu
portfolio.cov = shrinked_cov

frontier = portfolio.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)

frontier_5_10_sh = np.round(frontier.T,2)

In [163]:
frontier_performance(frontier_5_10_sh,mu,shrinked_cov)

,ISAC.L,^TBSP,XAUPLN,MWIG40TR,CORP.L,PLOPLN3M,Returns,Std. deviations,SR
0,0.00,0.0,0.0,0.0,0.0,1.00,0.0300,0.0037,8.11
1,0.12,0.0,0.0,0.0,0.0,0.88,0.0366,0.0173,2.12
2,0.23,0.0,0.0,0.0,0.0,0.77,0.0426,0.0307,1.39
3,0.34,0.0,0.0,0.0,0.0,0.66,0.0487,0.0441,1.10
4,0.45,0.0,0.0,0.0,0.0,0.55,0.0548,0.0575,0.95
5,0.56,0.0,0.0,0.0,0.0,0.44,0.0608,0.0710,0.86
6,0.67,0.0,0.0,0.0,0.0,0.33,0.0669,0.0844,0.79
7,0.78,0.0,0.0,0.0,0.0,0.22,0.0729,0.0978,0.75
8,0.89,0.0,0.0,0.0,0.0,0.11,0.0790,0.1113,0.71
9,1.00,0.0,0.0,0.0,0.0,0.00,0.0850,0.1247,0.68


In [167]:
acwi_tbsc = pln_returns[['ISAC.L','^TBSP']]

In [257]:
means = expected_returns[['ISAC.L','^TBSP']]
cov = acwi_tbsc.cov()
number_of_scenarios = 10001
T = 180
sim_returns = []

for s in range (number_of_scenarios):
    sim_returns.append(multivariate_normal.rvs(means,cov, T))

sim_returns = np.array(sim_returns)

In [258]:
sim_means = sim_returns.mean(axis=1)
sim_means.mean(0) *12

sim_std = sim_returns.std(axis=1)

sim_std.mean(0) * np.sqrt(12)

array([0.11474316, 0.04730829])

In [259]:
e = np.arange(0,1.1,0.1)
b = 1 - e
portfolios = np.vstack((e,b)).T


In [265]:
scenarios = sim_returns + 1
scenarios = np.cumprod(scenarios,axis=1)
r = 25
T = np.arange(11,180,12)

glide_path = []

for t in T:
    qv = []
    for p in portfolios:
        lw = scenarios[:,t] @ p
        qv.append(np.percentile(lw, r))
    glide_path.append(np.argmax(qv))

print(glide_path)
np.take(portfolios,glide_path,0)

[3, 4, 7, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]


array([[0.3, 0.7],
       [0.4, 0.6],
       [0.7, 0.3],
       [1. , 0. ],
       [1. , 0. ],
       [1. , 0. ],
       [1. , 0. ],
       [1. , 0. ],
       [1. , 0. ],
       [1. , 0. ],
       [1. , 0. ],
       [1. , 0. ],
       [1. , 0. ],
       [1. , 0. ],
       [1. , 0. ]])